<a href="https://colab.research.google.com/github/vietcode/colab/blob/zdownloader/ZDownloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title ZDownloader { display-mode: "form", form-width: "30%" }
#@markdown Path to a file containing links (1 link per line) to download, or a DLC file.
links = "/content/links.txt" #@param {type:"string"}
#@markdown Directory to download files to.
outdir = "/content/" #@param {type:"string"}
max_parallel_file_downloads = 20 #@param {type:"slider", min:1, max:20, step:1}
max_segments_per_file = 5 #@param {type:"slider", min:1, max:10, step:1}
#@markdown Password to extract if needed.
password = "" #@param {type:"string"}

import os
import requests 

# Install if haven't.
if not os.path.isdir("/content/.zdownloader"):
  !curl -L https://github.com/sin2000/zdownloader/releases/download/2.2/linux-x86-64.zip | jar xv
  !mv linux-x86-64 .zdownloader
  !chmod +x .zdownloader/start_zd.sh
  !chmod +x .zdownloader/zdownloader
  # Install crudini to edit .ini file of zdownloader
  !pip install iniparse # required by crudini.
  !curl https://raw.githubusercontent.com/pixelb/crudini/master/crudini > /usr/local/bin/crudini
  !chmod a+x /usr/local/bin/crudini

# Ensure outdir exists
!mkdir -p '{outdir}'
# Update configuration
!crudini --set /content/.zdownloader/zdownloader.ini 'Common' 'download_directory' '{outdir}'
!crudini --set /content/.zdownloader/zdownloader.ini 'Downloader' 'max_parallel_file_downloads' {max_parallel_file_downloads}
!crudini --set /content/.zdownloader/zdownloader.ini 'Downloader' 'max_segments_per_file' {max_segments_per_file}
!crudini --set /content/.zdownloader/zdownloader.ini 'UnpackPasswords' '1\pass' '{password}'

# Ensures the links file exists.
!touch '{links}'

# Decrypts DLC file into links.txt directly.
if links[-4:] == ".dlc":
  with open(links) as f:
    # Decrypt the DLC file for links.
    content = f.read()
    request = requests.post(url = "http://dcrypt.it/decrypt/paste", data = {"content": content})
    content = request.json()

    !touch "/content/.zdownloader/links.txt"
    with open("/content/.zdownloader/links.txt", "r+") as f:
      f.seek(0)
      f.write("\n".join(content["success"]["links"]))
      f.truncate()
# Otherwise, just copy the text file over.
else:
  !cp '{links}' /content/.zdownloader/links.txt

# Execute
!cd .zdownloader && ./start_zd.sh